## 라이브러리

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import tqdm as tqdm

In [ ]:
!pip install opencv-python
import cv2

## 이미지

In [ ]:
# !unzip archive.zip

In [ ]:
# 예측에 사용할 정답값을 폴더명을 사용해서 만들어 줍니다.
import os
root_dir = "dataset/"
image_label = os.listdir(root_dir)
image_label.remove("test.csv")
image_label

이미지 일부 미리보기

In [ ]:
import glob
fig, axes = plt.subplots(nrows=1, ncols=len(image_label), figsize=(20, 5))

for i, img_label in enumerate(image_label):
    wfiles = glob.glob(f"{root_dir}/{img_label}/*")
    wfiles = sorted(wfiles)
    img = plt.imread(wfiles[-1])
    axes[i].imshow(img)
    axes[i].set_title(img_label)

이미지 데이터셋 만들기

In [ ]:
#  이미지 파일을  array 로 만드는 과정은 어렵기 보다는 복잡합니다.
# * 복잡한 문제들이 대체적으로 어렵게 느껴져요.
# * 현업을 할 때도 업무가 복잡합니다. 과정을 이해하는게 중요합니다.
# * 작은 단위로 나눠 보는 것을 추천합니다.

# 0) 목표 train, valid, test set 에 대한 X, y값 만들기!
# 1) label 별로 각 폴더의 파일의 목록을 읽어옵니다. 
# 2) 이미지와 label  리스트를 만들어서 넣어줄 예정이에요.
# 3) test는 폴더가 따로 있어요. 이미지를 불러올 때  test 여부를 체크해서  train, test 를 먼저 만듭니다.
# 4) train 으로 train, valid 를 나누어 줍니다.

In [ ]:
def img_read_resize(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (120, 120))
    return img

In [ ]:
img_path = "dataset/cloudy/cloudy131.jpg"
# plt.imread(img_path).shape
img_read_resize(img_path).shape

In [ ]:
image_label

In [ ]:
x_train_img = []
x_test_img = []
y_train_img = []
y_test_img = []
# tqdm 을 통해 이미지를 읽어오는 상태를 표시합니다.
for img_label in tqdm.tqdm(image_label):
    img_files, labels = img_folder_read(img_label)
    if img_label != "alien_test":
        x_train_img.extend(img_files)
        y_train_img.extend(labels)
    else:    
        x_test_img.extend(img_files)
        y_test_img.extend(labels)

In [ ]:
len(x_train_img), len(x_test_img), len(y_train_img), len(y_test_img)

In [ ]:
x_train_img[0].shape

In [ ]:
a = [1, 2, 3, 4]
a_append = []
a_extend = []
a_append.append(a)
a_append.append(a)
a_extend.extend(a)
a_extend.extend(a)

In [ ]:
# append: 과자를 봉지째 넣어줍니다.
a_append

In [ ]:
# extend: 과자를 뜯어서 낱개로 넣어줍니다.
a_extend

In [ ]:
# len(x_train_img), len(y_train_img), x_train_img[0]

x, y 값 np.array 형식으로 만들기

In [ ]:
x_train_array  = np.array(x_train_img)
y_train_array  = np.array(y_train_img)
x_test_array  = np.array(x_test_img)
y_test_array  = np.array(y_test_img)

x_train_array.shape, y_train_array.shape, x_test_array.shape, y_test_array.shape

In [ ]:
# train_test_split
# x_train_raw, x_valid_raw, y_train_raw, y_valid_raw


In [ ]:
rand_no = np.random.choice(range(1, 1498), 5)
rand_no

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(12, 2))
for i, r_no in enumerate(rand_no):
    axes[i].imshow(x_train_array[r_no])
    axes[i].set_title(y_train_array[r_no])

train, valid 나누기

In [ ]:
for fdir in glob.glob(f"{root_dir}*"):
    print(fdir, len(glob.glob(f"{fdir}/*")))

In [ ]:
# train_test_split
from sklearn.model_selection import train_test_split
x_train_raw, x_valid_raw, y_train_raw, y_valid_raw = train_test_split(
    x_train_array, y_train_array, stratify=y_train_array, test_size=0.33, random_state=42)

In [ ]:
pd.Series(y_train_raw).value_counts(1)

In [ ]:
pd.Series(y_valid_raw).value_counts(1)

이미지 데이터 정규화

In [ ]:
# 정규화
x_train = x_train_raw / 255
x_valid = x_valid_raw / 255
x_test = x_test_array / 255

In [ ]:
np.min(x_train[0]), np.max(x_train[0])

In [ ]:
x_train.shape, x_valid.shape, x_test.shape

## 정답 One-Hot-Encoding

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit([1, 2, 6, 4, 2])

print(lb.classes_)
lb.transform([1, 6])

In [ ]:
# LabelBinarizer 를 사용하여 'cloudy', 'shine', 'sunrise', 'rainy', 'foggy' 형태의 분류를 숫자로 변경합니다.
# y_test 는 정답값 비교를 할 예정이고 학습에 사용하지 않기 때문에 인코딩 하지 않아도 됩니다.

lb = preprocessing.LabelBinarizer()
lb.fit(y_train_raw)
y_train = lb.transform(y_train_raw)
y_valid = lb.transform(y_valid_raw)

y_train.shape, y_valid.shape, lb.classes_

In [ ]:
y_train_raw[:5],  y_train[:5]

## 정답 빈도수

In [ ]:
lb.classes_

In [ ]:
# 예측값의 종류별 빈도수 입니다. 250~250 개 사이에 빈도수가 분포되어 있습니다.
pd.Series(np.argmax(y_train, axis=1)).value_counts(1)

## 층 구성

In [ ]:
# 환경변수 등록
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
num_classes = y_train.shape[1]
num_classes

In [ ]:
x_train[0].shape

In [ ]:
# model = Sequential()
# # 입력층
# model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=x_train[0].shape))
# model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(2,2))
          
# model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
# model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(2,2))
# model.add(Dropout(0.2))

# # Fully-connected layer
# model.add(Flatten())
# model.add(Dense(units=64, activation='relu'))
# # 출력층
# model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# model.trainable = True

# Fine-tuning을 위해 동결할 층 설정
for layer in vgg.layers:
    layer.trainable = False

# 일부 층 fine-tuning
model.layers[-3].trainable = True
model.layers[-2].trainable = True

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

vgg = VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=x_train[0].shape,
)
model = Sequential()
model.add(vgg)

# Fully-connected layer
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
# 출력층
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# summary
model.summary()

## Compile

In [ ]:
# compile
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=["accuracy"]
             )

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor="val_accuracy", patience=10, verbose=1)

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), 
                    epochs=100, callbacks=earlystop)

## history

In [ ]:
df_hist = pd.DataFrame(history.history)

In [ ]:
df_hist[["loss", "val_loss"]].plot()

## predict

In [ ]:
y_pred = model.predict(x_test)
y_predict = np.argmax(y_pred, axis=1)
y_predict

## 평가

In [ ]:
test = pd.read_csv("dataset/test.csv")
y_test = test["labels"]
test.head()

In [ ]:
# 예측값과 실제값을 비교해 봅니다.
(y_test == y_predict).mean()

In [ ]:
lb.classes_[0]

In [ ]:
fig, axes = plt.subplots(6, 5, figsize=(20, 20))
for i, xt in enumerate(x_test):
    col = i % 5
    row = i // 5
    color = "b"
    if y_test[i] != y_predict[i]:
        color = "r"
    axes[row][col].imshow(xt)
    axes[row][col].set_title(f"{test.loc[i, 'Image_id']}, {lb.classes_[y_predict[i]]}", c=color)